In [1]:
#####################################################################################
#########                                                              #############
#########   Establishing the four Random Forest Model for understang   ##############
########## the synergy between SIF, reflectance and vegetation index   #############
#########                                                              #############
####################################################################################

In [2]:

# Import required libraries

import warnings
warnings.filterwarnings('ignore')

import glob
from datetime import datetime, timedelta 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from scipy.stats import linregress
import scipy
import math
from math import sqrt

from sklearn.metrics import mean_squared_error, explained_variance_score, mean_absolute_error
from sklearn.metrics import precision_score, recall_score, roc_auc_score, roc_curve
from sklearn.metrics import r2_score

import statsmodels.formula.api as smf
import statsmodels as sm

%matplotlib inline
sns.set_style('whitegrid')


In [3]:
# Controling the label, font, axes and legend sizes
#plt.rc('font', size=16, weight='bold') #controls default text sizesns.set_style('whitegrid')
plt.rc('font', size=16) #controls default text sizesns.set_style('whitegrid')
plt.rc('xtick', labelsize=20) #fontsize of the x tick labels
plt.rc('ytick', labelsize=20) #fontsize of the y tick labels
plt.rc('legend', fontsize=20) #fontsize of the legend
plt.rc('axes', titlesize=20) #fontsize of the title
plt.rc('axes', labelsize=20) #fontsize of the x and y labels


In [4]:
# Set up your working directory
import os
os.getcwd()
os.chdir("D:/SIF_GPP_PRI_Tropomi/Linear_Regression_output/LR_Stats/LAST_testing/figures")

In [5]:
# load your merged data set

df_merge = pd.read_csv('D:\LCC/DataGPPfiltered.csv')

# convert your datetime into datetime format and set it as index 
df_merge['Timestamp'] = pd.to_datetime(df_merge['Timestamp'], format ='%m/%d/%Y')

#Filtering based on the distance (<=5 km) and cloud fraction (<=15%)
df_merge = df_merge[(df_merge['distance']<= 5)&(df_merge['cloud_fraction']<=0.15)]

# Outliers filtering
df_merge = df_merge[(df_merge['Err_mesure']<=0.15)&(df_merge['Err_mesure']>=-0.15)]

# Drop your columns 
df_merge['Timestamp'] = df_merge.set_index(df_merge['Timestamp'], inplace = True)
df_merge = df_merge.drop(columns = ['Timestamp'])

# Convert categorical variables as numeric, Biomes and Sites.
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

# label_encoder object
label_encoder =LabelEncoder()
# Encode labels in column. 
df_merge['IGBP_site']= label_encoder.fit_transform(df_merge['Biome_site'])
df_merge['Site']= label_encoder.fit_transform(df_merge['Site_name'])

#Rename daily averaged SIF
df_merge.rename(columns={'daily_averaged_SIF':'SIF_Daily'}, inplace = True)

# convert site name to liste and sort the data frame
listSites = df_merge["Site_name"].unique().tolist()
listSites = sorted(listSites)

# replace null GPP values as nan
df_merge['GPP'][df_merge['GPP']==0] = np.nan

#Drop nan values 
df_merge.dropna(subset = ['GPP'], inplace = True)

#get dayofyear from dataframe index
df_merge['DOY'] = df_merge.index.dayofyear

df_merge


,sif,sif_err,sif_relative,dcSIF,cloud_fraction,sza,vza,phase_angle,daily_correction_factor,lat,...,NEE_VUT_REF,PPFD_OUT,TA_F,VPD_F,Year,DoY,saveGPP,IGBP_site,Site,DOY
Timestamp,,,,,,,,,,,,,,,,,,,,,
2018-02-07,-0.1047,0.2426,-0.4586,-0.0265,0.0000,66.599998,24.200001,-78.300003,0.2535,51.301701,...,NaN,NaN,NaN,NaN,2018,737098,1.574183,5,0,38
2018-02-14,0.4536,0.2607,1.6363,0.1202,0.0000,64.400002,50.799999,91.500000,0.2651,51.308800,...,NaN,NaN,NaN,NaN,2018,737105,1.546119,5,0,45
2018-02-16,0.2810,0.2599,1.0219,0.0769,0.0450,64.199997,15.300000,-58.700001,0.2737,51.270401,...,NaN,NaN,NaN,NaN,2018,737107,1.833989,5,0,47
2018-02-16,0.0293,0.2675,0.0986,0.0080,0.0056,64.300003,15.300000,-58.799999,0.2736,51.332802,...,NaN,NaN,NaN,NaN,2018,737107,1.833989,5,0,47
2018-02-18,0.3418,0.2953,0.8903,0.0924,0.0196,62.900002,30.600000,-78.599998,0.2702,51.337101,...,NaN,NaN,NaN,NaN,2018,737109,1.639252,5,0,49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-25,0.3064,0.3227,0.9896,0.0956,0.0433,65.500000,35.400002,85.400002,0.3118,64.242798,...,125.159621,41.86,7.238958,0.580417,2020,738059,4.102004,3,39,269
2020-10-03,-0.0848,0.2930,-0.3594,-0.0248,0.0000,68.699997,2.600000,-67.500000,0.2926,64.250198,...,NaN,NaN,NaN,NaN,2020,738067,3.078926,3,39,277
2020-10-03,0.0474,0.2963,0.1945,0.0139,0.0000,68.699997,2.300000,-67.599998,0.2925,64.238403,...,NaN,NaN,NaN,NaN,2020,738067,3.078926,3,39,277


In [8]:
# generate a new data set based on the correlation matrix results to establish the RF models

columns = ['B1','B2','B3','B4','B5','B6','B7','B8','B9','B11','B13',
           'SIF_Daily', 'GPP', 'NDVI', 'NIRv','PRI13','IGBP_site','Year','day','DoY','Site_name',
           'Site_palette','Biome_site']

Data = df_merge[columns]
#Data = FR_Fon[columns]
Data= Data.dropna(axis =0)
#Data = Data[(Data['IGBP_LC5km']=='CRO')]
Data

,B1,B2,B3,B4,B5,B6,B7,B8,B9,B11,...,NDVI,NIRv,PRI13,IGBP_site,Year,day,DoY,Site_name,Site_palette,Biome_site
Timestamp,,,,,,,,,,,,,,,,,,,,,
2018-02-07,434,2080,249.0,491,2611,1530,852,217,338.0,388,...,0.655,0.261998,-0.090,5,2018,737097,737098,BE-Bra,BE-Bra,MF
2018-02-14,506,2213,239.0,539,2834,1843,1113,214,333.0,495,...,0.628,0.267298,-0.091,5,2018,737104,737105,BE-Bra,BE-Bra,MF
2018-02-16,506,2213,239.0,539,2834,1843,1113,214,333.0,495,...,0.628,0.267298,-0.091,5,2018,737106,737107,BE-Bra,BE-Bra,MF
2018-02-16,506,2213,239.0,539,2834,1843,1113,214,333.0,495,...,0.628,0.267298,-0.091,5,2018,737106,737107,BE-Bra,BE-Bra,MF
2018-02-18,414,1755,185.0,386,1864,1386,646,813,1133.0,924,...,0.618,0.208607,0.026,5,2018,737108,737109,BE-Bra,BE-Bra,MF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-25,153,470,127.0,191,398,188,104,1110,1023.0,906,...,0.509,0.043399,0.093,3,2020,738058,738059,SE-Svb,SE-Svb,ENF
2020-10-03,153,470,127.0,191,398,188,104,1110,1023.0,906,...,0.509,0.043399,0.093,3,2020,738066,738067,SE-Svb,SE-Svb,ENF
2020-10-03,153,470,127.0,191,398,188,104,1110,1023.0,906,...,0.509,0.043399,0.093,3,2020,738066,738067,SE-Svb,SE-Svb,ENF


In [11]:
# preparation for the random forest model setting
#get the required libraries 
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

#select your explanatory variables

columns = ['B1','B2','B3','B4','B5','B6','B7','B8','B9','B11','B13',
           'SIF_Daily', 'NDVI', 'NIRv','PRI13','IGBP_site','Year','day','DoY','Site_name','Biome_site']

# select your target variable
columntarg = ['GPP']
data = Data[columns]
#data['B15'] =data['B15'].astype(float)
target = Data[columntarg]
# split your dataset into training set and testing set: 80% for training and 20% for testing the model 
data_train, data_test, target_train, target_test = train_test_split(data, target, test_size= 0.20, random_state= 42)

#################################################################################################################################
#create a new data frame for testing data as you will need it to test your model not only on all data pooled across all sites but also based on each site and each PFT
df = pd.concat([data_test, target_test], axis =1)

#df.to_csv('data_testing2.csv') #uncomment this line to save your testing data in your working directory.

In [12]:

%%time

from sklearn.model_selection import train_test_split
#############################################################################################################################
#### Establishing the four RF models    ###################################################################################
##########################################################################################################################
columns = ['B1','B2','B3','B4','B5','B6','B7','B8','B9','B11','B13'] # uncomment this line to run your RF-R model
#columns = ['SIF_Daily','B1','B2','B3','B4','B5','B6','B7','B8','B9','B11','B13'] # uncomment this line to run your RF-SIF-R model
#columns = ['SIF_Daily','B1','B2','B3','B4','B5','B6','B7','B8','B9','B11','B13', 'IGBP_site']
#columns = ['SIF_Daily','NIRv','NDVI','PRI13']  # # uncomment this line to run your RF-SIF-VI model

######################################################################################################################
columntarg = ['GPP']
data = Data[columns]
target = Data[columntarg]

data_train, data_test, target_train, target_test = train_test_split(data, target, test_size= 0.20, random_state= 42)


## Importing Random Forest regressor from the sklearn.ensemble
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()

## Use Randomized SearchCV to tune your model parameters

from sklearn.model_selection import RandomizedSearchCV

Randomized_GridCV = {'bootstrap': [True, False],# method used to sample data points
            'max_depth': [5, 6, 7, 8, 9,10, 15, 20, 30, 40, 50, 60,
                        70, 80, 90, 100, 110,120], # maximum number of levels allowed in each decision tree
            'max_features': ['log2', 'sqrt','auto'], # number of features in consideration at every split
            'min_samples_leaf': [1, 2, 3, 4, 5, 10], # minimum sample number that can be stored in a leaf node
            'min_samples_split': [1, 2, 6, 10], # minimum sample number to split a node
            'n_estimators': [2, 5, 10, 15, 20, 20, 30, 50, 80, 100, 150, 200]}, # number of trees in the random forest

rf_random = RandomizedSearchCV(cv=10, estimator=RandomForestRegressor(), n_iter=20,
                   n_jobs=-1,
                   param_distributions= Randomized_GridCV,
                   random_state= 42, verbose=2)
# Fit your random forest model

rf_random.fit(data_train, target_train)

# print the best parameters
print ('Best Parameters: ', rf_random.best_params_, ' \n')

# use the best parameters for prediction

best_model = rf_random.best_estimator_

GPP_RF = best_model.predict(data_test)
print("RF_model_score=%.6f"% best_model.score(data_train, target_train))


Fitting 10 folds for each of 20 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   13.7s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:  4.3min finished


Best Parameters:  {'n_estimators': 10, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 40, 'bootstrap': False}  

RF_model_score=0.911012
Wall time: 4min 22s


In [23]:

%%time

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
sns.set_style('whitegrid')

#############################################################################################################################
#### Establishing the four RF models    ###################################################################################
##########################################################################################################################
columns = ['B1','B2','B3','B4','B5','B6','B7','B8','B9','B11','B13'] # uncomment this line to run your RF-R model
#columns = ['SIF_Daily','B1','B2','B3','B4','B5','B6','B7','B8','B9','B11','B13'] # uncomment this line to run your RF-SIF-R model
#columns = ['SIF_Daily','B1','B2','B3','B4','B5','B6','B7','B8','B9','B11','B13', 'IGBP_site']
#columns = ['SIF_Daily','NIRv','NDVI','PRI13']  # # uncomment this line to run your RF-SIF-VI model

######################################################################################################################

columntarg = ['GPP']
data = Data[columns]
#data['B15'] =data['B15'].astype(float)
target = Data[columntarg]

data_train, data_test, target_train, target_test = train_test_split(data, target, test_size= 0.20, random_state= 42)

## Importing Random Forest Classifier from the sklearn.ensemble
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()

from sklearn.model_selection import RandomizedSearchCV

# use the best parameters to make model prediction
best_model = RandomForestRegressor(n_estimators = 200, min_samples_split = 10, min_samples_leaf= 1, max_features = 'sqrt', max_depth= 40, bootstrap= False) 
#best_model = RandomForestRegressor(n_estimators = 200, min_samples_split = 2, min_samples_leaf= 1, max_features = 'sqrt', max_depth= 110, bootstrap=False) 


best_model.fit(data_train, target_train)


GPP_RF = best_model.predict(data_test)

print("RF_Prediction_score=%.6f"% best_model.score(data_test, target_test))
print("RF_model_score=%.6f"% best_model.score(data_train, target_train))
#print("params:", regr_rf.estimator_params)
#print("alpha:", regr_rf.ccp_alpha)

# Model Evaluation Parameters

MSE = mean_squared_error(target_test,GPP_RF)
RMSE = math.sqrt(MSE)
MAE = mean_absolute_error(target_test, GPP_RF)
EVS = explained_variance_score(target_test, GPP_RF, multioutput='variance_weighted')
R_squared = best_model.score(data_test, target_test)

# Calculate the adjusted R-squared
adjusted_R = 1 - (1-best_model.score(data_test, target_test))*(len(target_test)-1)/(len(target_test)-data_test.shape[1]-1)

print("Adjusted R_squared:", adjusted_R)
print("RMSE:",RMSE)
print("MAE:", MAE)
print("Explained_variance_score:",EVS )

target_test['GPP_RF_R'] = best_model.predict(data_test)

#target_test.to_csv('GPP_Predicted_RF_R.csv') # uncomment this line to record your model GPP prediction in your working directory.


RF_Prediction_score=0.861017
RF_model_score=0.912749
Adjusted R_squared: 0.8607038891651133
RMSE: 1.6796073552656978
MAE: 0.9412311967326992
Explained_variance_score: 0.8610316748006382
Wall time: 18.7 s


In [24]:
%%time

#######################################################################################################
########### Determining the feature relative importance        #######################################
######################################################################################################
# Get your feature_importances and mean permutation importances from the best model RF fit

#importances = best_model.feature_importances_
importances = best_model.feature_importances_

indices = np.argsort(importances)
features = data_train.columns
imp = []

dat = pd.DataFrame()
imp.append(importances)
########################################################################################################################

Dat= pd.DataFrame(imp).transpose()

#Dat = pd.concat([dfm, dfmean, std], axis =1)
Dat.columns = ['Importances']
Dat.index = data_train.columns
Dat.index.name ='Variables'

#rename columns 
#Dat.rename(columns ={'PRI13':'PRI', 'IGBP_LCnum':'IGBP_Biome'})
Dat.rename(index={'PRI13':'PRI','IGBP_site':'IGBP_Biome','SIF_Daily':'SIF Daily', 'GPP':'GPP'}, inplace = True)
Dat.reset_index(inplace = True)

Dat.sort_values(by= ['Importances',], inplace= True )

#print(Dat.index.name)
#Dat.to_csv('RF_R_Importances.csv') # uncomment this line to record your feature relative importance into your working directory
Dat

Wall time: 147 ms


,Variables,Importances
7,B8,0.047085
8,B9,0.055081
9,B11,0.064165
2,B3,0.067674
6,B7,0.069444
3,B4,0.070338
5,B6,0.076966
4,B5,0.081959
10,B13,0.098853
0,B1,0.131944
